# Project: Brazilian experiment Data Analysis 
By Ali AMIN

In [2]:
import pandas as pd
import numpy as np

In [5]:
qos_data = pd.read_csv('QoS.csv')
qos_data = qos_data.drop(qos_data.columns[[3, 4, 5,10,11,15]], axis=1) 
qos_data = qos_data.sort_values(by=['Session'],ascending=[True],axis=0)


qos_data.groupby("Session").describe().T


Session                       1           2           3           4  \
Annoyance     count  155.000000  124.000000  141.000000  159.000000   
              mean     3.716129    3.483871    3.290780    3.534591   
              std      1.210103    1.405471    1.411883    1.241579   
              min      0.000000    0.000000    0.000000    0.000000   
              25%      3.000000    2.000000    2.000000    3.000000   
              50%      4.000000    4.000000    4.000000    4.000000   
              75%      5.000000    5.000000    4.000000    5.000000   
              max      5.000000    5.000000    5.000000    5.000000   
Dif_interests count  155.000000  124.000000  141.000000  159.000000   
              mean     0.703226    0.362903    0.808511    0.635220   
              std      1.554943    1.375362    1.878126    1.729935   
              min     -5.000000   -3.000000   -4.000000   -4.000000   
              25%      0.000000    0.000000    0.000000    0.000000   
              50%      0.000000    0.000000    1.000000    1.000000   
              75%      2.000000    1.000000    2.000000    2.000000   
              max      5.000000    5.000000    5.000000    5.000000   
English       count  155.000000  124.000000  141.000000  159.000000   
              mean     0.135484    0.000000    0.000000    0.113208   
              std      0.343349    0.000000    0.000000    0.317847   
              min      0.000000    0.000000    0.000000    0.000000   
              25%      0.000000    0.000000    0.000000    0.000000   
              50%      0.000000    0.000000    0.000000    0.000000   
              75%      0.000000    0.000000    0.000000    0.000000   
              max      1.000000    0.000000    0.000000    1.000000   
Id            count  155.000000  124.000000  141.000000  159.000000   
              mean   558.696774  595.435484  643.241135  592.484277   
              std    308.568740  321.099768  323.885604  315.637548   
              min     21.000000   29.000000   30.000000   31.000000   
              25%    297.500000  332.250000  349.000000  329.000000   
              50%    560.000000  617.000000  689.000000  588.000000   
...                         ...         ...         ...         ...   
Stress        std      2.331242    2.551476    2.542904    2.387189   
              min      0.000000    0.000000    0.000000    0.000000   
              25%      6.000000    5.000000    5.000000    5.000000   
              50%      8.000000    7.000000    7.000000    7.000000   
              75%      9.000000    9.000000    9.000000    8.000000   
              max     10.000000   10.000000   10.000000   10.000000   
User_Id       count  155.000000  124.000000  141.000000  159.000000   
              mean   237.574194  252.766129  272.957447  251.094340   
              std    132.809157  138.194638  139.704580  136.694652   
              min      1.000000    1.000000    1.000000    1.000000   
              25%    127.500000  152.000000  157.000000  150.500000   
              50%    243.000000  269.500000  296.000000  253.000000   
              75%    344.000000  366.750000  395.000000  362.500000   
              max    476.000000  478.000000  479.000000  477.000000   
intHighQoS    count  155.000000  124.000000  141.000000  159.000000   
              mean     2.451613    2.895161    2.134752    2.509434   
              std      1.612426    1.518550    1.512896    1.363484   
              min      0.000000    0.000000    0.000000    0.000000   
              25%      1.000000    2.000000    1.000000    1.500000   
              50%      2.000000    3.000000    2.000000    3.000000   
              75%      4.000000    4.000000    3.000000    3.000000   
              max      5.000000    5.000000    5.000000    5.000000   
intLowQoS     count  155.000000  124.000000  141.000000  159.000000   
              mean     3.154839    3.258065    2.943262    3.144654   
              std      1

In [18]:
print(qos_data.info())
qos_data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1031 entries, 0 to 1030
Data columns (total 14 columns):
Id                 1031 non-null int64
Session            1031 non-null int64
Explanation_en     959 non-null object
Annoyance          1031 non-null int64
intLowQoS          1031 non-null int64
intHighQoS         1031 non-null int64
Comment_en         380 non-null object
Email              925 non-null object
English            1031 non-null int64
Session_key        852 non-null object
Preferred_video    1031 non-null int64
User_Id            1031 non-null int64
Stress             1031 non-null int64
Dif_interests      1031 non-null int64
dtypes: int64(10), object(4)
memory usage: 112.8+ KB
None


,Id,Session,Annoyance,intLowQoS,intHighQoS,English,Preferred_video,User_Id,Stress,Dif_interests
count,1031.000000,1031.000000,1031.000000,1031.000000,1031.000000,1031.000000,1031.000000,1031.000000,1031.000000,1031.000000
mean,597.290980,4.071775,3.545102,3.300679,2.750727,0.056256,1.558681,253.056256,6.825412,0.549952
std,325.933489,2.028125,1.329885,1.416542,1.511425,0.230527,0.498736,140.612353,2.425707,1.618815
min,21.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,-5.000000
25%,310.500000,2.000000,3.000000,2.000000,2.000000,0.000000,1.000000,140.000000,5.000000,0.000000
50%,608.000000,4.000000,4.000000,4.000000,3.000000,0.000000,2.000000,266.000000,8.000000,0.000000
75%,870.500000,6.000000,5.000000,4.000000,4.000000,0.000000,2.000000,370.500000,8.000000,2.000000
max,1153.000000,7.000000,5.000000,5.000000,5.000000,1.000000,2.000000,479.000000,10.000000,5.000000
